# Kayak Project

## Imports

In [56]:
import pandas as pd
import numpy as np
import plotly.express as px
import requests
from dotenv import load_dotenv
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import requests
import time
import boto3

load_dotenv()
%load_ext dotenv
%dotenv

# Openweathermap API
key = os.getenv('APIKEY')
# RDS Connection
DBHOST = os.getenv("DBHOST")
DBUSER = os.getenv("DBUSER")
DBPASS = os.getenv("DBPASS")
DBNAME = os.getenv("DBNAME")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [57]:
# Importing the cities list
cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [58]:
# Creating a dataframe with the cities list
df_cities = pd.DataFrame(columns=["city"])
df_cities['city'] = cities
df_cities.reset_index(inplace=True)
df_cities.rename(columns={'index': 'id'}, inplace=True)
df_cities

,id,city
0,0,Mont Saint Michel
1,1,St Malo
2,2,Bayeux
3,3,Le Havre
4,4,Rouen
5,5,Paris
6,6,Amiens
7,7,Lille
8,8,Strasbourg
9,9,Chateau du Haut Koenigsbourg


## Scraping Booking.com

In [4]:
# Creating a copy of the dataframe
df_booking = df_cities.copy(deep=True)

# Defining the spider class
class BookingSpider(scrapy.Spider):
    name = "Booking_data"
    cities = df_booking["city"]
    start_urls = ['https://www.booking.com/index.fr.html']

    def parse(self, response):
        for i in cities:
            yield scrapy.FormRequest.from_response(
            response,
            formdata={'ss': i },
            callback=self.after_search
        )
    # Important note: the classes names change often and may not be correct at a later date
    def after_search(self, response):
        cities = response.url.split("ss=")[-1].split("&")[0]
                
        booking = response.css('.d4924c9e74')
        
        for k in booking:
            
            yield {
                'location': cities,
                'name': k.css('a div.fcab3ed991.a23c043802::text').getall(),
                'url': k.css('h3.a4225678b2 a::attr(href)').getall(),
            }

        try:
            next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
            logging.info('No next page. Terminating crawling process.')
        else:
            yield response.follow(next_page, callback=self.after_search)

In [5]:
# Initializing the crawler process
filename = "cities.json"

if filename in os.listdir():
        os.remove(filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename: {"format": "json"},
    },
    "AUTOTHROTTLE_ENABLED": True
})

process.crawl(BookingSpider)
process.start()

2022-04-14 22:32:20 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-04-14 22:32:20 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.9.11 (main, Mar 29 2022, 19:08:29) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1n  15 Mar 2022), cryptography 36.0.0, Platform Linux-5.13.0-39-generic-x86_64-with-glibc2.31
2022-04-14 22:32:20 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) '
               'Gecko/20100101 Firefox/92.0'}
2022-04-14 22:32:20 [scrapy.extensions.telnet] INFO: Telnet Password: fae05b61e55cb43f
2022-04-14 22:32:20 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogS

In [10]:
# Reading the scraped json data into a new dataframe
df = pd.read_json("cities.json")
df.head()

,location,name,url
0,Mont+Saint+Michel,"[Le Relais Saint Michel, La Mère Poulard, Merc...",[https://www.booking.com/hotel/fr/le-relais-sa...
1,St+Malo,"[Studio cocooning, Apartment, St Malo, Petite ...",[https://www.booking.com/hotel/fr/studio-cocoo...
2,Bayeux,"[Premiere Classe Bayeux, ibis budget Bayeux, L...",[https://www.booking.com/hotel/fr/premiere-cla...
3,Le+Havre,"[Holiday Inn Express - Le Havre Centre, Ibis B...",[https://www.booking.com/hotel/fr/campanile-le...
4,Rouen,"[Maison hypercentre Rouen tout confort, L'Aute...",[https://www.booking.com/hotel/fr/maison-hyper...


In [11]:
# Keeping only the first 20 items from each list
for i in range (len(df["location"])):
    df["name"][i] = df["name"][i][:20]
    df["url"][i] = df["url"][i][:20]


In [13]:
# Initializing the scraper with BeautifulSoup
lat_full = []
lon_full = []
description_full = []
score_full = []

navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'

for i in range(len(df["url"])):
    lat_list = []
    lon_list = []
    description_list = []
    score_list = []
    hotel_list = df["url"][i]

    for j in hotel_list:

        # In some cases, the process will return an error on the first attempt
        try:
            page = requests.get(j, headers={'User-Agent': navigator})
            soup = BeautifulSoup(page.text, 'html.parser')
        except:
            page = requests.get(j, headers={'User-Agent': navigator})
            soup = BeautifulSoup(page.text, 'html.parser')

        lat_list.append(soup.select('p.address.address_clean a')[0].get("data-atlas-latlng").split(",")[0])
        lon_list.append(soup.select('p.address.address_clean a')[0].get("data-atlas-latlng").split(",")[1])
        description_list.append(soup.select('div#property_description_content')[0].get_text())

        try:
            score_list.append(soup.select('div.b5cd09854e.d10a6220b4')[0].get_text())
            
        except:
            # If the score is missing, setting the default value to 5.0
            score_list.append("5.0")

        # Setting up a wait timer to avoid spamming the website
        time.sleep(1.4)
        
    lat_full.append(lat_list)
    lon_full.append(lon_list)
    description_full.append(description_list)
    score_full.append(score_list)
    
    # Printing the progress, the whole process takes a bit more than 40 minutes
    print (f"{df['location'].iloc[i]} completed")

# Adding new columns based on lists of lists gathered above    
df["lat"] = lat_full
df["lon"] = lon_full
df["description"] = description_full
df["score"] = score_full
print('Task completed')

city Mont+Saint+Michel done
city St+Malo done
city Bayeux done
city Le+Havre done
city Rouen done
city Paris done
city Amiens done
city Lille done
city Strasbourg done
city Chateau+du+Haut+Koenigsbourg done
city Colmar done
city Eguisheim done
city Besancon done
city Dijon done
city Annecy done
city Grenoble done
city La+Rochelle done
city Bayonne done
city Biarritz done
city Montauban done
city Toulouse done
city Ariege done
city Carcassonne done
city Collioure done
city Saintes+Maries+de+la+mer done
city Aigues+Mortes done
city Nimes done
city Uzes done
city Avignon done
city Aix+en+Provence done
city Marseille done
city Cassis done
city Bormes+les+Mimosas done
city Gorges+du+Verdon done
city Lyon done


In [ ]:
# Replacing the + in cities names and reindexing the dataframe to add the id column that will be useful to merge later
df["location"] = df["location"].str.replace("+", " ", regex=True)
df = df.set_index("location")
df = df.reindex(cities)
df.reset_index(inplace=True)
df.reset_index(inplace=True)
df.rename(columns={'index': 'id'}, inplace=True)

## Getting API Data

### GPS Coordinates

In [59]:
# Creating another copy of the cities dataframe to store the coordinates data from Nominatim API
df_gps = df_cities.copy(deep=True)
lat_list = []
lon_list = []

for i in cities:
    # There is no Gorges du Verdon city, replacing by the closest city instead
    if i == "Gorges du Verdon":
        i = "La%20Palud-sur-Verdon"
        r = requests.get(f"https://nominatim.openstreetmap.org/search?city={i}&format=json").json()
        lat_list.append(r[0]['lat'])
        lon_list.append(r[0]['lon'])
    # Ariege is not a city, using the county instead here
    elif i == 'Ariege':
        r = requests.get(f"https://nominatim.openstreetmap.org/search?county={i}&format=json").json()
        lat_list.append(r[0]['lat'])
        lon_list.append(r[0]['lon'])
    # Replacing the whitespace by a web friendly notation
    else:
        name = i.replace(" ", "%20")
        r = requests.get(f"https://nominatim.openstreetmap.org/search?city={name}&format=json").json()
        lat_list.append(r[0]['lat'])
        lon_list.append(r[0]['lon'])
 
# Adding the coordinates to the dataframe   
df_gps['lat'] = lat_list
df_gps['lon'] = lon_list 

In [60]:
df_gps.head()

,id,city,lat,lon
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


### Weather Data

In [64]:
# Getting the weather data based on coordinates from the Openweathermap API
df_full = df_gps.copy(deep=True)
temps_list = []
rain_pop = []
humidity_list = []
# The days list will be used to store data as items of a list for day +1 to day +7
days = list(range(1,8))

for i in df_full.itertuples():
    lat = i.lat
    lon = i.lon
    r = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&units=metric&appid={key}").json()
    weather_7_days = r['daily'][1:] # Getting the weather data for the next 7 days, first item is the current weather, which we don't want here
    temps = [j['feels_like']['day'] for j in weather_7_days]
    rain = [int(j['pop'] * 100) for j in weather_7_days]
    humidity = [j['humidity'] for j in weather_7_days]
    temps_list.append(temps)
    rain_pop.append(rain)
    humidity_list.append(humidity)
    
df_full['day_plus'] = [days for _ in range(len(df_full))]
df_full['felt_temperature'] = temps_list
df_full['rain_chances'] = rain_pop
df_full['humidity'] = humidity_list
# Calculation of a weather quality indicator, using absolute value of 35 - temperature (with a weight of 2), chances of rain as a percentage, and humidity as a percentage (with a weight of 0.5) - lower is better
df_full['score_weather'] = df_full.apply(lambda x: abs((35 - np.mean(x['felt_temperature'])) * 2) + np.mean(x['rain_chances']) + (np.mean(x['humidity']) / 2), axis=1)   

In [112]:
df_full.head()

,id,city,lat,lon,day_plus,felt_temperature,rain_chances,humidity,score_weather
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,"[1, 2, 3, 4, 5, 6, 7]","[15.83, 15.06, 15, 15.98, 13.56, 15.26, 14.15]","[43, 24, 49, 100, 86, 45, 24]","[59, 48, 60, 66, 72, 61, 67]",123.974286
1,1,St Malo,48.649518,-2.0260409,"[1, 2, 3, 4, 5, 6, 7]","[12.29, 13.22, 13.46, 13.7, 12.37, 12.76, 12.97]","[37, 23, 60, 98, 76, 38, 8]","[79, 59, 69, 75, 76, 74, 72]",128.637143
2,2,Bayeux,49.2764624,-0.7024738,"[1, 2, 3, 4, 5, 6, 7]","[12.54, 12.48, 11.93, 15.25, 12.8, 12.83, 12.12]","[10, 0, 0, 97, 89, 0, 19]","[71, 61, 71, 72, 79, 65, 67]",109.728571
3,3,Le Havre,49.4938975,0.1079732,"[1, 2, 3, 4, 5, 6, 7]","[13.34, 12.66, 12.82, 14.04, 10.5, 11.9, 12.36]","[0, 0, 0, 100, 75, 25, 18]","[68, 56, 67, 75, 94, 67, 61]",110.965714
4,4,Rouen,49.4404591,1.0939658,"[1, 2, 3, 4, 5, 6, 7]","[14.93, 13.59, 14.52, 15.67, 10.08, 13.9, 13.95]","[0, 0, 0, 100, 56, 55, 7]","[58, 53, 63, 77, 93, 59, 55]",106.245714


## Merging Data

In [ ]:
# Renaming coordinates columns to avoid confusion between city and hotel coordinates
df.rename(columns={'lat': 'lat_hotels', 'lon': 'lon_hotels'}, inplace=True)

In [58]:
# Merging the hotels and weather dataframes, and dropping the unnecessary location column
df_complete = pd.merge(df_full, df, on='id')
df_complete.drop(columns=['location'], inplace=True)
print(df_complete.shape)

# Saving the merged dataset as a csv file
df_complete.to_csv('df_complete.csv', index=False, header=True)

## Storing the dataset in a S3 data lake

In [10]:
# Loading the S3 dession and resource
session = boto3.Session()
s3 = boto3.resource('s3')

In [11]:
# Uploading the file to S3
s3.Bucket('kayak-project-garp').upload_file('df_complete.csv', 'df_complete.csv')

In [6]:
# Downloading the file from S3 and checking it kept the same shape.
df_s3 = pd.read_csv('s3://kayak-project-garp/df_complete.csv')
df_s3.shape

(35, 15)

## Transforming

In [7]:
# Splitting the dataframe into 2 tables

df_weather = df_s3[['id', 'city', 'lat', 'lon', 'day_plus', 'felt_temperature', 'rain_chances', 'humidity', 'score_weather']]
df_hotels = df_s3[['id', 'city', 'name', 'url', 'lat_hotels', 'lon_hotels', 'description', 'score', 'score_weather']]

In [8]:
df_weather.head()

,id,city,lat,lon,day_plus,felt_temperature,rain_chances,humidity,score_weather
0,0,Mont Saint Michel,48.635954,-1.511460,"[1, 2, 3, 4, 5, 6, 7]","[15.48, 15.06, 14.02, 14.07, 15.12, 16.03, 15.88]","[36.0, 12.0, 100, 89.0, 25.0, 94.0, 100]","[60, 52, 73, 82, 63, 63, 79]",138.668571
1,1,St Malo,48.649518,-2.026041,"[1, 2, 3, 4, 5, 6, 7]","[12.25, 13.03, 13.19, 12.82, 13.44, 14.56, 13.83]","[24.0, 22.0, 100, 97.0, 28.000000000000004, 97...","[78, 68, 75, 81, 69, 68, 80]",147.322857
2,2,Bayeux,49.276462,-0.702474,"[1, 2, 3, 4, 5, 6, 7]","[12.14, 12.03, 13.79, 14.58, 15.8, 15.17, 15.88]","[4.0, 0, 98.0, 67.0, 22.0, 40.0, 91.0]","[75, 60, 67, 78, 70, 72, 79]",123.388571
3,3,Le Havre,49.493898,0.107973,"[1, 2, 3, 4, 5, 6, 7]","[13, 12.33, 14.04, 13.77, 12.5, 12.51, 14.59]","[0, 0, 93.0, 93.0, 47.0, 20.0, 81.0]","[70, 57, 62, 79, 85, 81, 83]",128.145714
4,4,Rouen,49.440459,1.093966,"[1, 2, 3, 4, 5, 6, 7]","[14.84, 13.33, 15.78, 16.45, 14.13, 16.72, 18.24]","[0, 0, 99.0, 100, 82.0, 44.0, 70.0]","[58, 54, 61, 72, 84, 65, 64]",127.860000


In [9]:
df_hotels.head()

,id,city,name,url,lat_hotels,lon_hotels,description,score,score_weather
0,0,Mont Saint Michel,"['Hôtel Vert', 'Le Relais Saint Michel', 'La M...",['https://www.booking.com/hotel/fr/vert.fr.htm...,"['48.61470049', '48.61758727', '48.63508532', ...","['-1.50961697', '-1.51039615', '-1.51053965', ...","[""\nVous pouvez bénéficier d'une réduction Gen...","['8,1', '7,8', '7,2', '8,2', '7,3', '7,2', '8,...",138.668571
1,1,St Malo,"['Hotel Eden', ""Hotel d'Aleth"", ""Hotel Ajoncs ...",['https://www.booking.com/hotel/fr/eden-saint-...,"['48.66190919', '48.63593081', '48.64735692', ...","['-1.98966533', '-2.02171236', '-2.02519655', ...","[""\nVous pouvez bénéficier d'une réduction Gen...","['7,3', '7,9', '8,5', '7,4', '9,2', '7,9', '9,...",147.322857
2,2,Bayeux,"['ibis budget Bayeux', 'Hôtel De Brunville et ...",['https://www.booking.com/hotel/fr/etap-bayeux...,"['49.25424209', '49.27815769', '49.26974024', ...","['-0.64648747', '-0.70351392', '-0.68883955', ...",['\nL’ibis budget Bayeux vous accueille entre ...,"['8,2', '7,8', '8,1', '9,8', '9,4', '8,7', '8,...",123.388571
3,3,Le Havre,"['Holiday Inn Express - Le Havre Centre', 'Nom...",['https://www.booking.com/hotel/fr/campanile-l...,"['49.49823800', '49.49331760', '49.49641693', ...","['0.12883700', '0.12530688', '0.15151234', '0....","[""\nL’Holiday Inn Express - Le Havre Centre, s...","['8,2', '8,6', '7,5', '8,7', '8,5', '7,2', '8,...",128.145714
4,4,Rouen,"['Radisson Blu Hotel, Rouen Centre', 'Best Wes...",['https://www.booking.com/hotel/fr/radisson-bl...,"['49.44644100', '49.44306423', '49.44332714', ...","['1.09412000', '1.08660311', '1.09457396', '1....","[""\nVous pouvez bénéficier d'une réduction Gen...","['8,9', '8,6', '8,2', '7,1', '8,2', '8,2', '7,...",127.860000


In [10]:
# The csv has converted the lists into strings, we need to revert that first for both dataframes
df_weather_final = pd.DataFrame()
df_weather_final[['id', 'city', 'lat', 'lon', 'score_weather']] = df_weather[['id', 'city', 'lat', 'lon', 'score_weather']]
df_weather_final['day_plus'] = df_weather['day_plus'].apply(eval)
df_weather_final['felt_temperature'] = df_weather['felt_temperature'].apply(eval)
df_weather_final['rain_chances'] = df_weather['rain_chances'].apply(eval)
df_weather_final['humidity'] = df_weather['humidity'].apply(eval)

df_hotels_final = pd.DataFrame()
df_hotels_final[['id', 'city', 'score_weather']] = df_hotels[['id', 'city', 'score_weather']]
df_hotels_final['name'] = df_hotels['name'].apply(eval)
df_hotels_final['url'] = df_hotels['url'].apply(eval)
df_hotels_final['lat_hotels'] = df_hotels['lat_hotels'].apply(eval)
df_hotels_final['lon_hotels'] = df_hotels['lon_hotels'].apply(eval)
df_hotels_final['description'] = df_hotels['description'].apply(eval)
df_hotels_final['score'] = df_hotels['score'].apply(eval)

# We need to convert coordinates to numeric type as well
df_weather_final[['lat', 'lon']] = df_weather[['lat', 'lon']].apply(pd.to_numeric)

In [11]:
# We need to sort the data by weather score and explode the lists to prepare the data for warehousing and plotting
df_weather_full = df_weather_final.sort_values('score_weather')
df_weather_full.reset_index(inplace=True, drop=True)
df_weather_full = df_weather_full.apply(pd.Series.explode)
df_weather_full[['day_plus', 'felt_temperature', 'rain_chances', 'humidity']] = df_weather_full[['day_plus', 'felt_temperature', 'rain_chances', 'humidity']].apply(pd.to_numeric)

df_hotels_full = df_hotels_final.sort_values('score_weather')
df_hotels_full.reset_index(inplace=True, drop=True)
df_hotels_full = df_hotels_full.apply(pd.Series.explode)
df_hotels_full['description'] = df_hotels_full['description'].replace("\\n", "", regex=True)
df_hotels_full['score'] = df_hotels_full['score'].replace(",", ".", regex=True)
df_hotels_full[['lat_hotels', 'lon_hotels', 'score']] = df_hotels_full[['lat_hotels', 'lon_hotels', 'score']].apply(pd.to_numeric)

In [12]:
df_hotels_full[:100]

,id,city,score_weather,name,url,lat_hotels,lon_hotels,description,score
0,7,Lille,82.091429,Hotel Lille Europe,https://www.booking.com/hotel/fr/lille-europe....,50.637888,3.072685,Vous pouvez bénéficier d'une réduction Genius ...,8.1
0,7,Lille,82.091429,Moxy Lille City,https://www.booking.com/hotel/fr/moxy-lille-ci...,50.627831,3.063592,"Le Moxy Lille City est situé à Lille, à 700 mè...",8.7
0,7,Lille,82.091429,ibis Styles Lille Centre Gare Beffroi,https://www.booking.com/hotel/fr/ibis-styles-l...,50.632382,3.067943,L’ibis Styles Lille Centre Gare Beffroi vous a...,7.8
0,7,Lille,82.091429,Hôtel Calm Lille,https://www.booking.com/hotel/fr/calm-lille.fr...,50.635288,3.069707,Vous pouvez bénéficier d'une réduction Genius ...,7.8
0,7,Lille,82.091429,"Holiday Inn Express Lille Centre, an IHG Hotel",https://www.booking.com/hotel/fr/expressbyholi...,50.630759,3.058088,L'Holiday Inn Express Lille Centre est situé d...,8.1
...,...,...,...,...,...,...,...,...,...
4,8,Strasbourg,116.914286,Hotel Rohan,https://www.booking.com/hotel/fr/hotel-de-roha...,48.580945,7.750431,L'Hôtel De Rohan est situé dans le centre hist...,8.7
4,8,Strasbourg,116.914286,Lagrange Apart’Hotel Strasbourg Wilson,https://www.booking.com/hotel/fr/lagrange-city...,48.587413,7.738243,Le Lagrange Apart’Hotel Strasbourg Wilson est ...,8.0
4,8,Strasbourg,116.914286,Aparthotel Adagio Access Strasbourg Petite France,https://www.booking.com/hotel/fr/adagio-access...,48.579248,7.732226,"Situé dans le centre-ville, l'Aparthotel Adagi...",8.4
4,8,Strasbourg,116.914286,Montempô Apparthôtel Strasbourg,https://www.booking.com/hotel/fr/montempo-appa...,48.579421,7.730113,Le Montempô Apparthôtel Strasbourg propose des...,6.8


## Storing the transformed data in a RDS data warehouse

In [13]:
# Importing libraries for the data warehousing and initialization of the SQL engine
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# Storing the two dataframes as SQL tables
df_weather_full.to_sql('weather', con=engine, if_exists='replace')
df_hotels_full.to_sql('hotels', con=engine, if_exists='replace')

In [ ]:
# Testing the data with a query
query = text("SELECT * FROM weather LIMIT 21")
pd.read_sql(query, engine)

In [ ]:
query = text("SELECT * FROM hotels LIMIT 40")
pd.read_sql(query, engine)

## Plotting Maps

In [38]:
# Creating a new column to be used to determine the size of the points in the plot
df_weather_full['inverted_score'] = max(df_weather_full['score_weather']) - df_weather_full['score_weather']

### Weather Map

In [47]:
# Creating a plotly scatter mapbox showing the 5 cities with the lowest weather score
px.set_mapbox_access_token(open(".mapbox_token").read())

fig = px.scatter_mapbox(
    df_weather_full[:35],
    lat='lat',
    lon='lon',
    color='felt_temperature',
    size='inverted_score',
    color_continuous_scale=px.colors.sequential.Bluered,
    size_max=35,
    zoom=4.7,
    range_color = [min(df_weather_full[:35]['felt_temperature']), max(df_weather_full[:35]['felt_temperature'])],
    hover_name='city',
    hover_data={
        'lat': False,
        'lon': False,
        'day_plus': False,
        'rain_chances': True,
        'humidity': True,
        'felt_temperature': True,
        'inverted_score': False,
        },
    animation_frame='day_plus'
)

fig.update_layout(
    width = 1100,
    height = 800,
    template='plotly_dark',
    title_x=0.5,
    title_text='The 5 cities with the best weather over the next 7 days')
fig.show()

### Hotels Map

In [55]:
# Creating a plotly scatter mapbox showing the 20 hotels from the previous 5 top cities
px.set_mapbox_access_token(open(".mapbox_token").read())

fig = px.scatter_mapbox(
    df_hotels_full[:100],
    lat='lat_hotels',
    lon='lon_hotels',
    color='score',
    color_continuous_scale=px.colors.diverging.RdYlGn,
    size='score',
    size_max=25,
    zoom=4.7,
    range_color = [min(df_hotels_full[:100]['score']), max(df_hotels_full[:100]['score'])],
    hover_name='city',
    hover_data={
        'lat_hotels': False,
        'lon_hotels': False,
        'name': True,
        'url': False,
        'description': False,
        'score_weather': False,
        'id': False,
        },
)

fig.update_layout(
    width = 1100,
    height = 800,
    template='plotly_dark',
    title_x=0.5,
    title_text='20 best hotels from the 5 cities with the best weather')
fig.show()